In [1]:
import sys
sys.path.append('../')

from typing import List
from faker import Faker
import jsonlines
import numpy as np

from elastic.elastic_api import ElasticIndex
from typing import Optional

In [4]:
def create_embedding():
    return np.random.uniform(low=-1.0, high=1.0, size=768).tolist()

def generate_random_docs(path_to_save="../data/documents.jsonl") -> List[int]:
    with jsonlines.open(path_to_save, mode="a") as writer:
        for i in range(100):
            sample = {}
            sample["doc_id"] = i
            sample["embedding"] = create_embedding()
            sample["all_fields"] = fake.name()
            sample["custom_hashtags"] = fake.name()
            sample["text_hashtags"] = fake.name()
            sample["photo_hashtags"] = fake.name()
            sample["photo_description"] = fake.name()
            sample["audio_transcription"] = fake.name()
            sample["song_name"] = fake.name()
            sample["song_author"] = fake.name()
            writer.write(sample)

generate_random_docs()

In [14]:
index_1 = ElasticIndex(index_name="test_1", 
                       path_to_index_json="../elastic/settings/index.json", 
                       elastic_host_port="8201", # Убедись что используешь правильный порт
                       elastic_password="Ne+QpMhnaPSmM+Lee5hd",
                       elastic_ca_certs_path="../elastic/certs/http_ca.crt",
                       path_to_documents="../data/documents.jsonl")

2024-06-08 01:11:53.685 | INFO     | elastic.elastic_api:create_index:35 - Index with name 'test_1' is created.


In [19]:
resp = index_1.local_client.search(index="test_1", query={"match_all": {}})
print("Got {} hits:".format(resp["hits"]["total"]["value"]))
for hit in resp["hits"]["hits"]:
    print(hit)
    break

Got 500 hits:
{'_index': 'test_1', '_id': 'jWTD9I8BF9rFQ-jgEpEm', '_score': 1.0, '_source': {'doc_id': 0, 'embedding': [0.5838319830797367, -0.684462736164865, -0.24043455114382262, 0.8123954630275052, -0.2591576909486104, 0.5017913967969985, -0.5883798526522062, -0.06745172320725379, -0.5151142260479449, 0.22912553185209794, -0.7541938036487923, 0.6539735594332872, 0.24771599989165582, 0.2049704699764594, 0.9569642489573091, 0.8376531356486427, -0.03257747938135358, 0.43767324658750195, -0.8526791024465574, -0.6111287083088417, -0.8335157600312204, -0.8936600236253722, -0.8454614525791011, -0.05250356492851349, 0.54209126569785, -0.2509041201860538, -0.5166613167395884, -0.9826962924961213, 0.21297111637605282, 0.44311315203064394, -0.9199505262618413, -0.2537742605780622, -0.7393072201593727, -0.26904939657044213, -0.03218688679801396, -0.9331908978000154, -0.9228682978890812, -0.7209237676007567, -0.7961022046236257, 0.053339324311334924, -0.34904626555505613, 0.08862132382015875, -

In [15]:
index_1.bulk_documents()

2024-06-08 01:12:55.559 | INFO     | elastic.elastic_api:bulk_documents:72 - Indexing documents... Overall documents: 500
100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 500/500 [00:01<00:00, 314.48docs/s]


In [16]:
index_1.count_documents_in_index()

2024-06-08 01:13:13.174 | INFO     | elastic.elastic_api:count_documents_in_index:53 - Count of documents: 500


In [ ]:
def elastic_query(index: ElasticIndex,
                  query: str,
                  explain: bool = False,
                  size: int = 20,
                  body: Optional[dict] = None):
    _explain = "true" if explain else "false"
    _query = str(query)
    _retailer_id = int(retailer_id)

    if body is None:
        body = {
            "_source": "false",
            "from": 0,
            "explain": _explain,
            "query": {
                "function_score": {
                    "query": {
                        "bool": {
                            "must": {
                                "dis_max": {
                                    "queries": [
                                        {
                                            "bool": {
                                                "filter": {
                                                    "term": {
                                                        "retailer_id": _retailer_id
                                                    }
                                                },
                                                "must": {
                                                    "multi_match": {
                                                        "fields": [
                                                            "custom_all^0.2",
                                                            "custom_all.morph^0.2",
                                                            "custom_all.synonyms^0.2"
                                                        ],
                                                        "operator": "AND",
                                                        "query": _query,
                                                        "type": "most_fields"
                                                    }
                                                },
                                                "should": {
                                                    "multi_match": {
                                                        "fields": [
                                                            "categories_filtered.synonyms^11.48",
                                                            "categories_filtered.morph^11.30",
                                                            "name.synonyms^11.27",
                                                            "name.morph^10.77",
                                                            "name^9.14",
                                                            "categories_filtered^9.07",
                                                            "master_category_id.synonyms^7.89",
                                                            "master_category_id.morph^7.77",
                                                            "ph_attr_sostav.synonyms^7.18",
                                                            "ph_attr_sostav.morph^7.02",
                                                            "ph_attr_sostav^6.05",
                                                            "master_category_id^5.14",
                                                            "root_categories.synonyms^5.02",
                                                            "root_categories.morph^4.89",
                                                            "root_categories^3.99",
                                                            "ph_attr_tip_produkta.synonyms^3.16",
                                                            "ph_attr_tip_produkta.morph^2.99",
                                                            "ph_attr_tip_produkta^2.87",
                                                            "ph_attr_tip_izdeliya.synonyms^1.61",
                                                            "ph_attr_tip_izdeliya.morph^1.58",
                                                            "ph_attr_tip_izdeliya^1.32",
                                                            "brand.synonyms^1.00",
                                                            "range_volume^1.00",
                                                            "range_fat^1.00",
                                                            "range_weight^1.00",
                                                            "brand^0.83",
                                                            "ph_attr_vkus.synonyms^0.39",
                                                            "ph_attr_vkus.morph^0.37",
                                                            "ph_attr_vkus^0.37",
                                                            "ph_attr_osnovnye_ingredienty.synonyms^0.35",
                                                            "ph_attr_osnovnye_ingredienty.morph^0.35",
                                                            "ph_attr_osnovnye_ingredienty^0.34"
                                                        ],
                                                        "operator": "AND",
                                                        "query": _query,
                                                        "type": "most_fields"
                                                    }
                                                }
                                            }
                                        },
                                        {
                                            "multi_match": {
                                                "fields": [
                                                    "name.prefix^10",
                                                    "name.prefix.*^10",
                                                    "categories.prefix^8",
                                                    "categories.prefix.*^8",
                                                    "brand.prefix^6",
                                                    "brand.prefix.*^6"
                                                ],
                                                "operator": "AND",
                                                "query": _query,
                                                "type": "bool_prefix"
                                            }
                                        }
                                    ]
                                }
                            }
                        }
                    }
                }
            },
            "size": size,
            "timeout": "500ms"
        }

    try:
        response = index.local_client.search(index=index.index_name, body=body)
    except Exception as e:
        # more smart exception can be here
        raise e

    return response
